# CLUSTERING DE SEMILLAS

## Introducción

En esta practica nos enfocamos en clustering. Clustering es una técnica de aprendizaje no supervidado para organizar en distintos grupos los datos de una semilla, de manera que los datos en un grupo tengan características similares.
Primero usaremos una herramienta llamada PCA para determinar cuál es el mejor scaler para usar con esta semilla y luego aplicaremos distintos modos de agrupamiento con el objetivo de encontrar los hiper-parametros más relevantes y discutir los resultados obtenidos.

#### Integrantes: Javier Díez y Mario Hidalgo